## Importación librerías:

In [130]:
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
from plotly import figure_factory as FF 

py.sign_in('mabellca', 'AWVznYF4d44wnIlS3Rrn') #claves plotly
accesstoken="pk.eyJ1Ijoic2FuenNhbmEiLCJhIjoiY2p1MHphZDFiMHBodjRlcDl6dWw3ZGZpbyJ9.U3ygIabawE-AxJ37lw_F4g" # clave mapbox

## Carga datos:

In [131]:
df = pd.read_csv('Datos_Totales.csv')
df.head()

,Unnamed: 0,Unnamed: 0.1,_id,date,favourites,language,latitude,longitude,retweets,tweet,user
0,0,0,5c9d5212dd43f237785f90c3,2019-03-28 23:00:28,0,pt,-10.943923,-37.084749,0,"Se você é real, sinceramente nem sei... o que ...",_Louise_Oficial
1,1,1,5c9d5233dd43f237785f90c5,2019-03-28 23:01:01,0,en,34.052200,-118.243000,0,#tbt to 12 year old Blake who was 5 feet tall ...,BlakePickens
2,2,2,5c9d5240dd43f237785f90c7,2019-03-28 23:01:15,0,en,33.448200,-112.077700,0,"just letting everyone know i'm in AZ , too @ P...",itselisawtf
3,3,3,5c9d5261dd43f237785f90c9,2019-03-28 23:01:47,0,en,40.810712,-73.962023,0,I remember when I could only do ONE singular p...,onlybryan_
4,4,4,5c9d5267dd43f237785f90cb,2019-03-28 23:01:53,0,pt,-23.100566,-47.204752,0,"Boa noite 😘😘😘 em Jardim do Sol, Indaiatuba htt...",Hanadaciro


## Acondicionamiento dataset:

Eliminamos las columnas que no nos interesan como los retweets, favourites, _id y modificamos la variable `date` de forma que sólo dejamos la hora, es decir, todos los registros recopilados a partir de los 00:01 de una hora concreta hasta los 59:59 de la misma hora se les asigna el mismo número (entre 1 y 48) en la nueva variable hora.

In [132]:
a=df.date.str.split(' ')
hora=a.str[1]

b=hora.str.split(':')
b.str[0]

hora=b.str[0]
hora=hora.astype(int)+1


dia=a.str[0].str.split('-').str[2].astype(int)

dff=df[['latitude','longitude','user','tweet']].copy()
dff['hora']=hora
dff['dia']=dia
dff.hora[dff.dia==28]=0
dff.dia[dff.dia==28]=0
dff.dia[dff.dia==29]=0
dff.dia[dff.dia==30]=1

dff.hora=dff.hora+dff.dia*24
dff.hora=dff.hora +1
dff=dff.drop('dia', axis=1)
dff=dff[:-3]
dff.tail()


,latitude,longitude,user,tweet,hora
7327,25.957696,-80.238801,walterleehui,#itautakemetothefinals @ Hard Rock Stadium htt...,48
7328,33.590416,130.399546,fujisuke_0826,"I'm at 天神地下街 - @tenjinchikagai in Fukuoka, 福岡県...",48
7329,14.574162,-90.550987,lklandaverde,#Flying en Iltra De Petapa https://t.co/JwPxZh...,48
7330,-19.377207,-40.053792,rgama7,Hummmmmm!!! Vai ter um encontro!!!!!\nanabeatr...,48
7331,46.484060,30.739350,vodaf22,Ежегодное о главном. #vodaf22 #deribasovskaya ...,48


## Código dashboard:

In [133]:

y = list(dff.groupby('hora').count().latitude)
x = list(np.arange(48)+1)

trace1 = {
  "type": "scattermapbox"
}

trace2=go.Scatter(
    x = x,
    y = y,
    mode = 'lines+markers',
    name = 'lines+markers')

sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Hora:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}
data = Data([trace1,trace2])
updatemenus = dict(

    # GENERAL
    direction="left",
    pad = dict(
        r=10,
        t=87),
    type = "buttons",
    showactive = False,
    x = 0.1,
    y = 0,
    xanchor = "right",
    yanchor = "top",

    # BUTTONS
    buttons=[
        dict(
            method = "animate",
            label = "Play",

            # PLAY
            args = [
                None,
                dict(
                    frame = dict(duration = 500, redraw = False), # False quicker but disables animations
                    fromcurrent = True,
                    transition = dict(duration = 300, easing = "quadratic-in-out"), # easing = "cubic-in-out"
                    ),
                ],
            ),
        dict(
            method = "animate",
            label = "Pause",

            # PAUSE
            args = [
                [None], # Note the list
                dict(
                    frame = dict(duration = 0, redraw = False), # Idem
                    mode = "immediate",
                    transition = dict(duration = 0),
                    ),
                ],
            )
        ],

)
layout = dict(

    title = "Tweets Map 48h",

    # GENERAL LAYOUT
    height = 1000,
    font = dict(
        color = "white",
    ),
    margin = dict(
        r=60,
        t=40,
        b=40,
        l=60
    ),
    showlegend = False,
    hovermode = "closest",

    # ANIMATIONS
    sliders = [sliders_dict],
    updatemenus = [updatemenus],

    # COLOR THEME
    plot_bgcolor = "black",
    paper_bgcolor = "black",

    # MAPBOX
    mapbox = dict(
        accesstoken = accesstoken,
        bearing=0,
        zoom = 1,
        pitch=0,
        style = "dark",
        domain = dict(
            y = [0.2,1]
        ),
    ),
    xaxis = dict(
        range = [x[0], x[47]],
        domain = [0, 1],
        anchor = "y4",
        title = "Hora",
    ),
    yaxis=dict(
    domain=[0,0.2],
    gridcolor="#444",
    title="Conteo Tweets",
    zerolinecolor="#444",
    )
)



frame=[]
for i in (np.arange(48)):
    
    frame1 = {
        "data": [
            {
              "hoverinfo": "lon+lat+text",
              "lat": list(dff[dff.hora==i+1].latitude), 
              "lon": list(dff[dff.hora==i+1].longitude),   
              "type": "scattermapbox",
              "text": list(dff[dff.hora==i+1].user)
            }, 
            {
              "x": [x[i]], 
              "y": [y[i]], 
              "marker": {
                "color": "rgb(0,107,204)", 
                "symbol": "circle",
                "size": 10,
                },
              "mode": "lines+markers", 
              "type": "scatter",
              "name":str(i)
            }
        ],
      'name': str(i),
      "traces": [0,1]
    }
    slider_step = {'args': [
        [i],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': int(i+1),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)
    frame.append(frame1)

fig = go.Figure(data=data, layout=layout, frames=frame)

py.icreate_animations(fig) 
